In [ ]:
! pip install -U spacy

In [ ]:
import spacy
import pandas as pd
from spacy.tokens import DocBin
from tqdm import tqdm
import random
import re

In [ ]:
! python -m spacy train ../input/models/config.cfg --output ./output_model --gpu-id 0

## Apply model

In [ ]:
def load_results_manual(X_text, visualize = True):
    """This function ouputs different labels, so the output os a dictionary"""
    nlp = spacy.load("./output_model/model-best")
    doc = nlp(X_text)
    if visualize == True:
        spacy.displacy.render(doc, style='ent', jupyter=True)
    ents = doc.ents
    output = []
    for entity in ents:
        row = {'entity': entity.text,
                'label': entity.label_,
                'start': entity.start,
                'end': entity.end}
        output.append(row)
    return output

## Submission

In [ ]:
test_ids = ['0FB0700DAF44', '18409261F5C2', 'D46BCB48440A', 'D72CB1C11673', 'DF920E0A7337']
txt_files = []
for i in test_ids:
    txt_file = open(f"../input/feedback-prize-2021/test/{i}.txt", "r")
    txt_file = txt_file.read()
    txt_files.append(txt_file)

In [ ]:
dfs = []
for j, item in enumerate(test_ids):
    entities = load_results_manual(txt_files[j], visualize=False)
    ids = [item for i in range(len(entities))]
    labels = [entities[i]['label'] for i in range(len(entities))]
    pred_string = []
    for i in range(len(entities)):
        start = int(entities[i]['start']) 
        end = int(entities[i]['end'] + 1)
        pred_string.append(list(range(start, end)))
    test_dict = {'id': ids, 'class': labels, 'predictionstring': pred_string}
    df_test = pd.DataFrame.from_dict(test_dict)
    dfs.append(df_test)

submission_df = pd.concat(dfs)
submission_df = submission_df.reset_index().drop(['index'], axis = 1)
submission_df.to_csv('./submission.csv')

In [ ]:
submission_df